## Real-Time Face Mask Detector with Python, OpenCV, Keras


I am going to build this project in two parts:

1. In the first part, I will write a python script using Keras to train face mask detector model. 

2. In the second part, I  test the results in a real-time webcam using OpenCV.


#### 1. Imports:
Import all the libraries and modules required.

In [1]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

Using TensorFlow backend.


#### 2. Build the neural network:

This convolution network consists of two pairs of Conv and MaxPool layers to extract features from the dataset.
Which is then followed by a Flatten and Dropout layer to convert the data in 1D and ensure overfitting.

And then two Dense layers for classification.

In [2]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


#### 3. Image Data Generation/Augmentation:

In [3]:
TRAINING_DIR = "Dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(224, 224))
VALIDATION_DIR = "Dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(224, 224))

Found 1315 images belonging to 2 classes.
Found 211 images belonging to 2 classes.


#### 4. Initialize a callback checkpoint to keep saving best model after each epoch while training:

In [4]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

#### 5. Train the model:

In [5]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10
132/132 [==============================] - 353s 3s/step - loss: 0.6300 - acc: 0.7118 - val_loss: 0.0352 - val_acc: 0.8626
Epoch 2/10
132/132 [==============================] - 356s 3s/step - loss: 0.3315 - acc: 0.8639 - val_loss: 0.0627 - val_acc: 0.9431
Epoch 3/10
132/132 [==============================] - 432s 3s/step - loss: 0.2531 - acc: 0.8981 - val_loss: 0.0339 - val_acc: 0.9242
Epoch 4/10
132/132 [==============================] - 428s 3s/step - loss: 0.2379 - acc: 0.9065 - val_loss: 0.0069 - val_acc: 0.9526
Epoch 5/10
132/132 [==============================] - 448s 3s/step - loss: 0.2305 - acc: 0.9156 - val_loss: 4.8959e-04 - val_acc: 0.9668
Epoch 6/10
132/132 [==============================] - 442s 3s/step - loss: 0.2144 - acc: 0.9118 - val_loss: 0.0426 - val_acc: 0.9668
Epoch 7/10
132/132 [==============================] - 441s 3s/step - loss: 0.1942 - acc: 0.9285 - val_loss: 0.0069 - val_acc: 0.9858
Epoch 8/10
132/132 [==============================] - 931s 7s/ste

In [6]:
#To save the trained model
model.save('mask_recog_ver1.h5')